## Data loading - Benchmark
Source: https://github.com/JessyFrish/YLFW_Links?tab=readme-ov-file

Make suitable for ArcFace and MagFace (112x112) and FaceNet (160x160)

### Data folder structure and metadata


In [1]:
# Load packages

# Common
import pandas as pd
import numpy as np
import os
import cv2 as cv
from IPython.display import clear_output as cls

# Data 
from glob import glob

# Data Visuaalization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Model



### Descriptive analysis of data

In [3]:
## Create dataframe 

# Define the path to the raw data directory
raw_data_path = '../data/raw/'

# Initialize lists to store image characteristics
img_names = []
ethnicities = []
genders = []
ages = []

# Iterate through the directories of each ethnicity (e.g., 'Asian')
ethnicities_list = os.listdir(raw_data_path + 'YLFW_bench/data_p2')
for ethnicity in ethnicities_list:
    ethnicity_dir = os.path.join(raw_data_path + 'YLFW_bench/data_p2', ethnicity)
    if os.path.isdir(ethnicity_dir):
        # Iterate through image files in the ethnicity directory
        for img_name in os.listdir(ethnicity_dir):
            
            # Append image characteristics to lists
            img_names.append(img_name)
            ethnicities.append(ethnicity)


# Create a DataFrame from the lists
data = {'img_name': img_names, 'ethnicity': ethnicities}
df = pd.DataFrame(data)

# Display the DataFrame
df.head()


,img_name,ethnicity
0,Asian_14_84.png,Asian_14
1,Asian_14_19.png,Asian_14
2,Asian_14_33.png,Asian_14
3,Asian_14_6.png,Asian_14
4,Asian_14_12.png,Asian_14
